In [1]:
from datetime import datetime
import polars as pl
import glob
from tqdm import tqdm

In [2]:
metadata = pl.read_csv('raw/metadata.csv', try_parse_dates=True)
metadata

user,start_date,end_date,length_days,length_years,potential_samples,actual_samples,missing_samples_abs,missing_samples_pct,contract_start_date,contract_end_date,contracted_tariff,self_consumption_type,p1,p2,p3,p4,p5,p6,province,municipality,zip_code,cnae
str,"datetime[μs, UTC]","datetime[μs, UTC]",f64,f64,i64,i64,i64,f64,date,date,str,str,f64,f64,f64,f64,f64,f64,str,str,i64,i64
"""00000c5a448d9faa097b761cc98036…",2022-05-30 01:00:00 UTC,2022-06-05 00:00:00 UTC,6.0,0.016427,144,144,0,0.0,2022-05-31,null,"""2.0TD""","""""",2.2,2.2,0.0,0.0,0.0,0.0,"""Gipuzkoa""",null,null,9329
"""0001b3b2f18c01c62ed9b2a87de7b4…",2017-05-31 01:00:00 UTC,2022-06-05 00:00:00 UTC,1831.0,5.013005,43944,43863,81,0.184326,2021-06-01,null,"""2.0TD""","""""",3.45,3.45,null,null,null,null,"""Bizkaia""",null,null,9820
"""0003de2700e20a1681d69fe287441d…",2017-05-31 01:00:00 UTC,2019-11-14 00:00:00 UTC,897.0,2.455852,21528,21476,52,0.241546,2019-09-02,2019-11-13,"""2.0A""","""""",4.6,null,null,null,null,null,"""Gipuzkoa""","""Donostia/San Sebastian""",20013,9820
"""0004150214d14a2b2e6f7075531e66…",2018-07-12 01:00:00 UTC,2022-06-05 00:00:00 UTC,1424.0,3.8987,34176,34169,7,0.020482,2021-06-01,null,"""2.0TD""","""""",9.2,9.2,null,null,null,null,"""Gipuzkoa""","""Irun""",20304,4759
"""000721f0fc6ccf02ae24b673939795…",2017-05-30 01:00:00 UTC,2022-06-04 00:00:00 UTC,1831.0,5.013005,43944,43815,129,0.293555,2021-06-01,null,"""2.0TD""","""""",4.4,4.4,null,null,null,null,"""Bizkaia""",null,null,9820
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""ffeddf4919e748beb50af7c6b73def…",2018-10-02 01:00:00 UTC,2021-11-03 00:00:00 UTC,1128.0,3.088296,27072,27069,3,0.011082,2021-06-01,null,"""2.0TD""","""""",6.0,6.0,null,null,null,null,"""Soria""",null,null,9820
"""ffefb9e27715b1c3fa5007c457de55…",2021-07-06 01:00:00 UTC,2022-06-05 00:00:00 UTC,334.0,0.914442,8016,8014,2,0.02495,2021-07-07,null,"""2.0TD""","""""",3.45,3.45,0.0,0.0,0.0,0.0,"""Bizkaia""","""Bilbao""",48012,9820
"""fff0adcd6cdb7434496a5222f03337…",2017-08-17 01:00:00 UTC,2022-06-05 00:00:00 UTC,1753.0,4.799452,42072,41102,970,2.305571,2021-06-01,null,"""2.0TD""","""""",4.6,4.6,null,null,null,null,"""Bizkaia""",null,null,9820


In [5]:
import pandas as pd

In [106]:
# Convert dates to timezone-aware datetime objects (UTC)
# metadata['start_date'] = pl.(metadata['start_date'], utc=True)
# metadata['end_date'] = pd.to_datetime(metadata['end_date'], utc=True)
candidate_periods = [
    [2016, 1, 1, 2019, 12, 31],
    [2016, 7, 1, 2019, 6, 30],
    [2017, 1, 1, 2019, 12, 31],
    [2017, 3, 1, 2020, 2, 29],
    [2017, 5, 1, 2020, 4, 30],
    [2017, 5, 15, 2020, 5, 16],
    [2017, 5, 25, 2020, 5, 26],
    [2017, 6, 1, 2020, 5, 31],
    [2017, 7, 1, 2020, 6, 30],
]

# candidate_periods = [
#     ("2016-01-01 00:00:00+00:00", "2019-12-31 23:59:59+00:00"),
#     ("2016-07-01 00:00:00+00:00", "2019-06-30 23:59:59+00:00"),
#     ("2017-01-01 00:00:00+00:00", "2019-12-31 23:59:59+00:00"),
#     ("2017-03-01 00:00:00+00:00", "2020-02-29 23:59:59+00:00"),
#     ("2017-05-01 00:00:00+00:00", "2020-04-30 23:59:59+00:00"),
#     ("2017-06-01 00:00:00+00:00", "2020-05-31 23:59:59+00:00"),
#     ("2017-07-01 00:00:00+00:00", "2020-06-30 23:59:59+00:00"),
# ]

In [107]:
# # Create DataFrame
# df_candidates = pl.DataFrame({
#     "start_datetime": [s for s, e in candidate_periods],
#     "end_datetime": [e for s, e in candidate_periods]
# })
#
# # Convert to datetime
# df_candidates = df_candidates.with_columns([
#     pl.col("start_datetime").str.strptime(pl.Datetime, format="%Y-%m-%d %H:%M:%S%z"),
#     pl.col("end_datetime").str.strptime(pl.Datetime, format="%Y-%m-%d %H:%M:%S%z"),
# ])
#
# print(df_candidates)

In [116]:
def calculate_total_hours(start_date, end_date):
    diff = end_date - start_date
    total_hours = diff.dt.total_days()
    return total_hours

# Function to evaluate coverage with fixed timezone handling
def evaluate_coverage(metadata_:pl.DataFrame, start_end, coverage_threshold=0.9):
    start, end = (pl.datetime(start_end[0],start_end[1],start_end[2], time_zone='UTC'),
                  pl.datetime(start_end[3],start_end[4],start_end[5], time_zone='UTC'))

    qualified_users = metadata_.filter(
        pl.col("start_date") <= start,
        pl.col("end_date") >= end)

    # n_qualified_users = qualified_users['']
    print(f"Qualified users {qualified_users['user'].unique().len()}")
    # print(qualified_users['user'].unique()[:10])
    return qualified_users
    # qualified_users.with_columns(total_hours.alias('expected_samples'))
    # qualified_users.with_columns((qualified_users['actual_samples'] / qualified_users['expected_samples']).alias('coverage'))
    # # qualified_users['coverage'] = qualified_users['actual_samples'] / qualified_users['expected_samples']
    # users_meeting_coverage = qualified_users[qualified_users['coverage'] >= coverage_threshold]

    # return len(users_meeting_coverage)

In [ ]:
# Re-evaluate each candidate period with corrected timezone
coverage_results = []
for start_end in candidate_periods:
    num_users = evaluate_coverage(metadata, start_end)
    # coverage_results.append({'qualified_users': num_users})
    # coverage_results.append(num_users)
# Convert results to DataFrame for clear comparison
# coverage_df = df_candidates.with_columns('qualified_users', coverage_results)

In [128]:
qualified_users = metadata.filter(
    pl.col("start_date") <= pl.datetime(2017, 6, 1, time_zone='UTC'),
    pl.col("end_date") >= pl.datetime(2020, 3, 1, time_zone='UTC'),
    (pl.col('cnae') > 9699),
    (pl.col('cnae') < 9849)
)

In [126]:
qualified_users.head()[0]

user,start_date,end_date,length_days,length_years,potential_samples,actual_samples,missing_samples_abs,missing_samples_pct,contract_start_date,contract_end_date,contracted_tariff,self_consumption_type,p1,p2,p3,p4,p5,p6,province,municipality,zip_code,cnae
str,"datetime[μs, UTC]","datetime[μs, UTC]",f64,f64,i64,i64,i64,f64,date,date,str,str,f64,f64,f64,f64,f64,f64,str,str,i64,i64
"""0001b3b2f18c01c62ed9b2a87de7b4…",2017-05-31 01:00:00 UTC,2022-06-05 00:00:00 UTC,1831.0,5.013005,43944,43863,81,0.184326,2021-06-01,null,"""2.0TD""","""""",3.45,3.45,null,null,null,null,"""Bizkaia""",null,null,9820


In [129]:
qualified_users['user'].unique().len()

4928